In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import evallm
from automata.fa.dfa import DFA
import tqdm.auto as tqdm

In [3]:
import itertools
from permacache import stable_hash

In [4]:
from evallm.prompting.transducer_prompt import (
    ChainOfThoughtPromptRealExampleNoExplanation,
    BasicSequencePrompt,
    BasicSequencePromptSlightlyMoreExplanation,
    BasicSequencePromptNoChat,
    SequencePromptWithExplanation,
    SequencePromptWithExplanationChainOfThought,
    RedGreenRoomPrompt1,
)
from evallm.experiments.transducer_experiment import (
    current_transducer_experiments,
    compute_relative_to_null,
    compute_relative_to_ngram,
    print_example,
    bottom_quartile_outcome,
    current_dfa_sample_spec,
)
from evallm.experiments.transducer_plotting import (
    plot_all_absolute_results_single_graph,
    plot_absolute_results_barchart,
)

In [5]:
from evallm.sample_dfa.sample_dfa import sample_dfa

In [6]:
from evallm.experiments.sequence_completion.sample_sequences import *
from evallm.experiments.sequence_completion.ngram_suffix_heuristic import *

In [7]:
def attempt(seed, num_sequence_symbols):
    spec = current_dfa_sample_spec(3)
    rng = np.random.RandomState(seed)
    dfa, sequences_prefixes = sample_sequence_completion_problem(
        rng,
        dfa_spec=spec,
        num_sequences=30,
        num_sequence_symbols=num_sequence_symbols,
        num_sequence_symbols_prompt=num_sequence_symbols // 2,
        num_instances=100,
        try_limit=50,
    )
    result = []
    for sequences, prefix in sequences_prefixes:
        out = ngram_heuristic(sequences, prefix)
        result += [dfa.accepts_input(prefix + out)]
    return np.mean(result)

In [8]:
np.mean([attempt(i, 8) for i in tqdm.trange(100)])

  0%|          | 0/100 [00:00<?, ?it/s]

np.float64(0.8292999999999999)

In [9]:
np.mean([attempt(i, 30) for i in tqdm.trange(100)])

  0%|          | 0/100 [00:00<?, ?it/s]

np.float64(0.8549)